In [1]:
import pandas as pd
import glob
from padelpy import padeldescriptor

In [7]:
df3 = pd.read_csv('data/cleaned_bioactivity_data.csv')
df3 = df3[['canonical_smiles', 'molecule_chembl_id']]

In [8]:
df3.to_csv('data/molecule.smi', sep='\t', index=False, header=False)

In [3]:
xml_files = glob.glob("data/xml_files/*.xml")
xml_files.sort()

In [4]:
FP_list = ['AtomPairs2DCount',
 'AtomPairs2D',
 'EState',
 'CDKextended',
 'CDK',
 'CDKgraphonly',
 'KlekotaRothCount',
 'KlekotaRoth',
 'MACCS',
 'PubChem',
 'SubstructureCount',
 'Substructure']

In [5]:
fp = dict(zip(FP_list, xml_files))

In [9]:
fingerprint = 'PubChem'

fingerprint_output_file = ''.join(['data/PubChem_fingerprints/',fingerprint,'.csv']) #Substructure.csv
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='data/molecule.smi', 
                d_file=fingerprint_output_file, #'Substructure.csv'
                #descriptortypes='SubstructureFingerprint.xml', 
                descriptortypes= fingerprint_descriptortypes,
                detectaromaticity=True,
                standardizenitro=True,
                standardizetautomers=True,
                threads=2,
                removesalt=True,
                log=True,
                fingerprints=True)

In [10]:
descriptors = pd.read_csv(fingerprint_output_file)